In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('snap.csv', sep='\t')
data.drop(columns=['Unnamed: 0','label'] , inplace=True)
data.head()

,comment,label_id
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,1.0
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,0.0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,1.0
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,0.0
4,شیرینی وانیلی فقط یک مدل بود.,0.0


In [3]:
data.info()
data.isnull().sum()
data.dropna(inplace=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   comment   70000 non-null  object 
 1   label_id  69480 non-null  float64
dtypes: float64(1), object(1)
memory usage: 1.1+ MB


In [4]:
data['label_id']=data['label_id'].astype(int)
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69480 entries, 0 to 69999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   comment   69480 non-null  object
 1   label_id  69480 non-null  int32 
dtypes: int32(1), object(1)
memory usage: 1.3+ MB


,comment,label_id
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,1
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,1
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,0
4,شیرینی وانیلی فقط یک مدل بود.,0


In [5]:
with open('persian',encoding="utf8") as stop_word_file:
    stop_words=stop_word_file.readlines()
    
stop_words = [line.replace('\n' , '') for line in stop_words]



In [6]:
max_len = 0
for index , row in data.iterrows():
    sentence_list=[]
    comment = row['comment']
    com_len = len(comment.split(' '))
    if com_len>max_len:
        max_len = com_len
    for word in comment.split():
        if word not in stop_words:
            sentence_list.append(word)
            
    data.loc[index]={
        'comment':' '.join(sentence_list),
        'label_id':row['label_id']
    }
    

In [8]:
data.head()


,comment,label_id
0,حیف وقت بنویسم سرویس دهیتون افتضاح,1
1,قرار ساعته برسه نیم ساعت موقع رسید، ببین چقدرر...,0
2,قیمت مدل کیفیتش سازگاری نداره، ظاهر فریبنده دا...,1
3,عالللی اندازه کیفیت خوب، امیداورم کیفیتتون باش...,0
4,شیرینی وانیلی مدل بود.,0


In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [61]:
vectorizer = TfidfVectorizer()
x = vectorizer.fit_transform(data['comment'])

In [62]:
y = data['label_id'].values

In [63]:
np.shape(x)
np.shape(y)

(69480,)

In [64]:
from sklearn import svm
from sklearn.model_selection import train_test_split



In [65]:
x_train , x_test, y_train , y_test = train_test_split(x,y)

In [66]:
svm_model = svm.SVC()
svm_model.fit(x_train , y_train)

SVC()

In [67]:
svm_model.score(x_test , y_test)

0.823258491652274

In [68]:
from sklearn.metrics import classification_report , confusion_matrix
y_pred = svm_model.predict(x_test)

In [69]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.88      0.76      0.81      8752
           1       0.78      0.89      0.83      8618

    accuracy                           0.82     17370
   macro avg       0.83      0.82      0.82     17370
weighted avg       0.83      0.82      0.82     17370



In [48]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Dropout , Dense , Flatten , Embedding , Conv1D , MaxPooling1D

In [30]:
x_train2 , x_test2, y_train2 , y_test2 = train_test_split(data['comment'],y)
keras_toknizer = Tokenizer()
x_nu = keras_toknizer.fit_on_texts(x_train2)

In [35]:
vocab_len = len(keras_toknizer.word_index) +1
keras_toknizer.word_index

{'غذا': 1,
 'بود': 2,
 'کیفیت': 3,
 'عالی': 4,
 'سفارش': 5,
 'پیتزا': 6,
 'سرد': 7,
 'خوشمزه': 8,
 'ارسال': 9,
 'طعم': 10,
 'ممنون': 11,
 'مرغ': 12,
 'پیک': 13,
 'بود،': 14,
 'سیب': 15,
 'تحویل': 16,
 'مزه': 17,
 'بسته': 18,
 'سس': 19,
 'نان': 20,
 'قیمت': 21,
 'گرم': 22,
 'رستوران': 23,
 'حجم': 24,
 'نسبت': 25,
 'اسنپ': 26,
 'زمینی': 27,
 'ساندویچ': 28,
 'نون': 29,
 'راضی': 30,
 'کیک': 31,
 'پایین': 32,
 'سلام': 33,
 'ساعت': 34,
 'شیرینی': 35,
 'سالاد': 36,
 'پنیر': 37,
 'گوشت': 38,
 'فود': 39,
 'موقع': 40,
 'دستم': 41,
 'مونده': 42,
 'کباب': 43,
 'باشه': 44,
 'نبود': 45,
 'خشک': 46,
 'مشتری': 47,
 'جوجه': 48,
 'برنج': 49,
 'خوبی': 50,
 'میشه': 51,
 'داغ': 52,
 'ممنونم': 53,
 'تاخیر': 54,
 'نظر': 55,
 'رسید': 56,
 'ریخته': 57,
 'غذای': 58,
 'دقیقه': 59,
 'میداد': 60,
 'تشکر': 61,
 'العاده': 62,
 'سوخته': 63,
 'خمیر': 64,
 'مواد': 65,
 'سوخاری': 66,
 'خرید': 67,
 'مناسب': 68,
 'توجه': 69,
 'قارچ': 70,
 'افتضاح': 71,
 'بشه': 72,
 'آب': 73,
 'بوی': 74,
 'سیر': 75,
 'برگر': 76,
 'جعبه': 7

In [36]:
#Prepare Data For Embbeding
encoded = keras_toknizer.texts_to_sequences(x_train2)

In [38]:
encoded[0]

[1, 55, 76, 4, 2]

In [39]:
padded_text = pad_sequences(encoded , maxlen=max_len , padding='post')

In [47]:
padded_text.shape

(52110, 361)

In [49]:
input1 = Input(shape=(padded_text.shape[1],))
embdding1 = Embedding(vocab_len,100)(input1)
conv1 = Conv1D(filters=32 , kernel_size=3 , activation='relu')(embdding1)
drop1 = Dropout(0.5)(conv1)
pool1 = MaxPooling1D(pool_size=2)(drop1)
flat1 = Flatten()(pool1)
dense1= Dense(10 , activation='relu')(flat1)
output= Dense(1 , activation='sigmoid')(dense1)

In [51]:
model = Model(inputs = input1 , outputs=output)

In [52]:
model.compile(loss = 'binary_crossentropy' , optimizer='adam' , metrics='accuracy')

In [53]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 361)]             0         
                                                                 
 embedding (Embedding)       (None, 361, 100)          3199500   
                                                                 
 conv1d (Conv1D)             (None, 359, 32)           9632      
                                                                 
 dropout (Dropout)           (None, 359, 32)           0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 179, 32)          0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 5728)              0         
                                                             

In [55]:
encoded_test = keras_toknizer.texts_to_sequences(x_test2)
padded_test = pad_sequences(encoded_test , maxlen=max_len , padding='post')


In [70]:
model.fit(padded_text , y_train2 , epochs=5 , batch_size=20 , validation_data=(padded_test , y_test2))

Epoch 1/5
2606/2606 [==============================] - 226s 87ms/step - loss: 0.1022 - accuracy: 0.9556 - val_loss: 0.9756 - val_accuracy: 0.7821
Epoch 2/5
2606/2606 [==============================] - 236s 91ms/step - loss: 0.0982 - accuracy: 0.9576 - val_loss: 1.0361 - val_accuracy: 0.7805
Epoch 3/5
2606/2606 [==============================] - 235s 90ms/step - loss: 0.0930 - accuracy: 0.9609 - val_loss: 1.0627 - val_accuracy: 0.7811
Epoch 4/5
2606/2606 [==============================] - 239s 92ms/step - loss: 0.0906 - accuracy: 0.9610 - val_loss: 1.1674 - val_accuracy: 0.7761
Epoch 5/5
2606/2606 [==============================] - 252s 97ms/step - loss: 0.0884 - accuracy: 0.9620 - val_loss: 1.1493 - val_accuracy: 0.7769
